<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/Multi-Agent-AI-model-DnD-Character-/blob/main/multi_agent_compound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent Compound AI (DnD Character)

This project aims to create a multi-agent compound AI that can take a classic novel/story as an input, separate the story into scenes based on locations, identify any characters within the scene relevant to the plot and use this information to create a character profile and corresponding artwork. The idea is that this tool could be used to create Dungeons & Dragons campaigns based on classic novels. This would be a basic proof of concept effort in this regard.

In [1]:
#Install dependencies
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5"

In [4]:
#Login to huggingface so we can push the trained model to the hub later

from huggingface_hub import notebook_login

notebook_login()

In [2]:
#Import libraries
import torch
from transformers import pipeline
import os

## 1. Divide into Scenes

The first step is to use an agent to divide the story text into scenes based on location. In a DnD game, players progress through the story by moving from one location to the next. They can also return to locations, but that would constitute a separate scene.

In [ ]:
#Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:512"

In [5]:
#Load and test model (Dolly 2.0 - 7B)

generate_text = pipeline(model="databricks/dolly-v2-7b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/13.8G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
res[0]["generated_text"]

"Nuclear fission and fusion are both processes that occur in the nucleus of an atom. Nuclear fission is the splitting of an atom's nucleus, leading to the release of more particles than were present in the original atom. This occurs when a fissioning isotope (one with an odd number of protons) absorbs a neutron from somewhere inside the nucleus of the atom and becomes a more stable isotope (one with an even number of protons). At the same time, another atom with the same isotope, but with an additional neutron, absorbs a proton from within its nucleus and fissions also, releasing more energy than the original atom had when it was first formed. \n\nFusion is the process by which two or more atomic nuclei come together and combine, or merge, to form a single, larger nucleus. In the process of fusion, two different nuclei can react together and form two atoms of the same element, or two different elements can react and form two different elements, neither of which is fissile. Fusion is us

In [ ]:
f = open("./alice_in_wonderland.txt", "r")

text = f.read()

prompt = f"""### Instruction:
Separate the provided input below into scenes based on location. Each time the location changes in the story, a new scene starts. Provide a a 50 word summary of each scene.

### Input:
{text}

"""

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)
pipeline("Hey how are you doing today?")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
403 Client Error. (Request ID: Root=1-668c0dd5-7ea7d3557bba25556b9196b0;40312f05-60e9-4a1b-885d-8cee152b4af6)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Your request to access model meta-llama/Meta-Llama-3-8B is awaiting a review from the repo authors.